In [42]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn  as sns
from datetime import datetime

abs_path = os.path.abspath("")

## 510k

In [43]:
'''Searching functions are ready, there are 10 parameters for filtering: ['KNUMBER', 'APPLICANT', "COUNTRY_CODE", "DATERECEIVED", 'DECISIONDATE', "DECISION", 'PRODUCTCODE', "STATEORSUMM", "THIRDPARTY", 'DEVICENAME']. Only  "DATERECEIVED", 'DECISIONDATE' take list as parameters, [year, mode]'''

'Searching functions are ready, there are 10 parameters for filtering: [\'KNUMBER\', \'APPLICANT\', "COUNTRY_CODE", "DATERECEIVED", \'DECISIONDATE\', "DECISION", \'PRODUCTCODE\', "STATEORSUMM", "THIRDPARTY", \'DEVICENAME\']. Only  "DATERECEIVED", \'DECISIONDATE\' take list as parameters, [year, mode]'

In [44]:
data_file_list = ["pmnlstmn.txt", "pmn96cur.txt", "pmn9195.txt", "pmn8690.txt", "pmn8185.txt", "pmn7680.txt"]
data_list = []
for i in range(len(data_file_list)):
    path = os.path.join(abs_path, "FDA Registration Record", "510k", data_file_list[i])
    data_list.append(pd.read_csv(path, sep= "|", encoding="cp1252"))

DATA = pd.concat(data_list)
DATA = DATA.reset_index(drop=True)


In [45]:
FDA_INFO_510K = DATA[['KNUMBER', 'APPLICANT', "COUNTRY_CODE", "DATERECEIVED", 'DECISIONDATE', "DECISION", 'PRODUCTCODE', "STATEORSUMM", "THIRDPARTY", 'DEVICENAME']]
FDA_INFO_510K = FDA_INFO_510K.rename({'APPLICANT':'MANUFACTURER', "DEVICENAME":"MODELNAME"}, axis="columns")
FDA_INFO_510K = FDA_INFO_510K.replace(np.nan, "NA", regex=True)
FDA_INFO_510K['DATERECEIVED'] = pd.to_datetime(FDA_INFO_510K['DATERECEIVED'], errors = "coerce")
FDA_INFO_510K['DECISIONDATE'] = pd.to_datetime(FDA_INFO_510K['DECISIONDATE'], errors = "coerce")

In [46]:
import datetime as dt

In [47]:
import datetime as dt
filter_510k_kNumber = lambda df, knumber : df[df["KNUMBER"].str.lower() == knumber.lower()]
filter_510k_manufacturer = lambda df, manufacturer : df[df["MANUFACTURER"].str.lower().str.contains(manufacturer.lower())]
filter_510k_countryCode = lambda df, countryCode : df[df["COUNTRY_CODE"].str.lower() == countryCode.lower()]
# wanted_period = [(%YYYY, %MM, %DD), (%YYYY, %MM, %DD)] -> [start_date, end_date], included
def filter_510k_dateReceived(df, wanted_period):
    start_date_tuple = wanted_period[0]
    end_date_tuple = wanted_period[1]
    start_date = dt.date(start_date_tuple[0], start_date_tuple[1], start_date_tuple[2])
    end_date = dt.date(end_date_tuple[0], end_date_tuple[1], end_date_tuple[2])
    df_a = df[df['DATERECEIVED'].dt.date >= start_date]
    df_b = df[df['DATERECEIVED'].dt.date <= end_date]
    return df_a.merge(df_b, how = 'inner', indicator = False) 
# wanted_period = [(%YYYY, %MM, %DD), (%YYYY, %MM, %DD)] -> [start_date, end_date], included
def filter_510k_decisionDate(df, wanted_period):
    start_date_tuple = wanted_period[0]
    end_date_tuple = wanted_period[1]
    start_date = dt.date(start_date_tuple[0], start_date_tuple[1], start_date_tuple[2])
    end_date = dt.date(end_date_tuple[0], end_date_tuple[1], end_date_tuple[2])
    df_a = df[df['DECIDSIONDATE'].dt.date >= start_date]
    df_b = df[df['DECIDSIONDATE'].dt.date <= end_date]
    return df_a.merge(df_b, how = 'inner', indicator = False) 
filter_510k_decision = lambda df, decision : df[df["DECISION"].str.lower() == decision.lower()]
filter_510k_productCode = lambda df, productCode : df[df["PRODUCTCODE"].str.lower() == productCode.lower()]
filter_510k_stateOrSum = lambda df, stateOrSum : df[df["STATEORSUMM"].str.lower() == stateOrSum.lower()]
filter_510k_thirdParty = lambda df, thirdParty : df[df["THIRDPARTY"].str.lower() == thirdParty.lower()]
filter_510k_modelName = lambda df, modelName: df[df["MODELNAME"].str.lower().str.contains(modelName.lower())]

func_list = [filter_510k_kNumber, filter_510k_manufacturer, filter_510k_countryCode, filter_510k_dateReceived, filter_510k_decisionDate, filter_510k_decision, filter_510k_productCode, filter_510k_stateOrSum, filter_510k_thirdParty, filter_510k_modelName]

In [48]:
def filter_510k(df, knumber = None, manufacturer = None, countryCode = None, dateReceived = None, decisionDate = None, decision = None, productCode = None, stateOrSum = None, thirdParty = None, ModelName = None):
    input_list = [knumber, manufacturer, countryCode, dateReceived, decisionDate, decision, productCode, stateOrSum, thirdParty, ModelName]
    result_df = df
    for i in range(len(input_list)):
        if input_list[i] != None:
            result_df = func_list[i](result_df, input_list[i])
    return result_df

In [57]:
result = filter_510k(FDA_INFO_510K, dateReceived = [(2018,3,9), (2019,5,7)])
result

,KNUMBER,MANUFACTURER,COUNTRY_CODE,DATERECEIVED,DECISIONDATE,DECISION,PRODUCTCODE,STATEORSUMM,THIRDPARTY,MODELNAME
0,K190217,"Aidite (Qinhuangdao) Technology Co., Ltd.",CN,2019-02-04,2020-12-17,SESE,EBG,Summary,N,Aidite PMMA
1,K190217,"Aidite (Qinhuangdao) Technology Co., Ltd.",CN,2019-02-04,2020-12-17,SESE,EBG,Summary,N,Aidite PMMA
2,K190217,"Aidite (Qinhuangdao) Technology Co., Ltd.",CN,2019-02-04,2020-12-17,SESE,EBG,Summary,N,Aidite PMMA
3,K190217,"Aidite (Qinhuangdao) Technology Co., Ltd.",CN,2019-02-04,2020-12-17,SESE,EBG,Summary,N,Aidite PMMA
4,DEN180012,"Intellirod Spine, Inc.",US,2018-03-14,2019-03-28,DENG,QFP,NA,N,LOADPRO™ Intraoperative Rod Strain Sensor
...,...,...,...,...,...,...,...,...,...,...
3434,K191222,"Prismatik Dentalcraft, Inc.",US,2019-05-07,2019-08-05,SESE,NHA,Summary,N,Inclusive Titanium Abutments compatible with: ...
3435,K191223,"Blue Belt Technologies, Inc.",US,2019-05-07,2019-06-28,SESE,OLO,Summary,N,NAVIO
3436,K191225,"Rhythmlink International, LLC",US,2019-05-07,2019-08-05,SESE,GXY,Summary,N,EEG Electrode Template
3437,K191226,Arthrex Inc.,US,2019-05-07,2019-06-11,SESE,MAI,Summary,N,Arthrex SwiveLock Suture Anchor
